In [33]:
from openai import OpenAI
import re

base_model_host = "nid008316"
instruct_model_host = "nid008313"
instruct_model_host = "nid008424"

model_base = 'allenai/OLMo-2-1124-7B'
model_ins = 'allenai/OLMo-2-1124-7B-Instruct'
model_ins = 'meta-llama/Llama-3.1-8B-Instruct'

client_base = OpenAI(base_url=f'http://{base_model_host}:8001/v1', api_key="EMPTY")
client_ins = OpenAI(base_url=f'http://{instruct_model_host}:8001/v1', api_key="EMPTY")
# client_llm = OpenAI(base_url=f'http://{llm_host}:8001/v1', api_key="EMPTY")

# model_llm = cmdline_args.llm_name

prompt_simple = "You are a helpful assistant. When responding to questions, enclose your thoughts in <thought></thought> and your clean, final answer in <answer></answer>. The answer should be a short phrase or name rather than a sentence."
prompt_gen_read = prompt_simple + " When you are asked a factual question about an entity, you must try to recollect by yourself the wikipedia article about the entity first without using any external tools."

def get_answers(question, model=None, temperature=1.0, max_completion_tokens=1024, gen_read=False, n=1, augment=None):
    if model is None: model = model_ins
    user = question if not augment else f'{question}\nIf it helps, here is what you recollected earlier about the entity/concept in question:\n```{augment}```'
    print(user)
    output = client_ins.chat.completions.create(model=model, messages=[{
                "role": "system",
                "content": prompt_simple if not gen_read else prompt_gen_read
            }, {'role': 'user', 'content': user}], max_completion_tokens=max_completion_tokens, temperature=temperature, n=n)
    return [c.message.content for c in output.choices]

def extract_answer(solution_str):
    answer = re.findall(r'<answer>(.*?)</answer>', solution_str, re.IGNORECASE)
    if answer:
        return answer[0]
    else:
        return ""


In [40]:
from concurrent.futures import ThreadPoolExecutor
import json
import tqdm
from datasets import load_dataset

out_dir = f'generations/memo/fifty_fifty/split{0}'

k_sample = 32 # cmdline_args.n
def collect_answers(ans):
    for i, ans_groups in tqdm.tqdm(enumerate(ans)):
        for j, ans_group in enumerate(ans_groups):
            for k, a in enumerate(ans_group):
                ans[i][j][k] = a.result()
freq = '100000_to_inf' #'0_to_1000'# '0_to_1000' #'0_to_1000', '1000_to_10000', '10000_to_100000', '100000_to_inf']
dataset = load_dataset('parquet', data_files=f"../{out_dir}/{freq}/dev.decon.wikilinked.prefixes.parquet")['train']
ref_aug_answers = []
extraction_success = []
for i in [0]:#tqdm.tqdm(range(len(dataset))):
    instance = dataset[i]
    ref_aug_answers_groups = []
    for doc_group, ref_aug in zip(instance['s_docs'], instance['prefix_before_answer']):
        ref_aug_answers_groups.append(get_answers(instance['question'], augment=doc_group['text'][:2048*4], n=k_sample))
        assert False
    ref_aug_answers.append(ref_aug_answers_groups)
# ref_aug_answers = [json.dumps([ri for ri in r]) for r in ref_aug_answers]
# dataset = dataset.add_column('prefix_before_answer_extraction_success', extraction_success)
# dataset = dataset.add_column('model_ref_doc_augmented_predictions', ref_aug_answers)
# dataset.to_parquet(f'{out_dir}/{freq}/dev.wikilinked.qa.ref_doc.parquet')


What is the capital of Bolivia?
If it helps, here is what you recollected earlier about the entity/concept in question:
```Bolivia

Bolivia, officially the Plurinational State of Bolivia, is a landlocked country located in western-central South America. It is bordered by Brazil to the north and east, Paraguay to the southeast, Argentina to the south, Chile to the southwest and Peru to the west. The seat of government and executive capital is La Paz, while the constitutional capital is Sucre. The largest city and principal industrial center is Santa Cruz de la Sierra, located on the Llanos Orientales (tropical lowlands), a mostly flat region in the east of the country.
The sovereign state of Bolivia is a constitutionally unitary state, divided into nine departments. Its geography varies from the peaks of the Andes in the West, to the Eastern Lowlands, situated within the Amazon basin. One-third of the country is within the Andean mountain range. With of area, Bolivia is the fifth larges

AssertionError: 

In [41]:
[extract_answer(ans) for ans in ref_aug_answers_groups[0]]

['La Paz and Sucre',
 'La Paz',
 'La Paz and Sucre',
 'La Paz and Sucre',
 'La Paz and Sucre ',
 'La Paz.',
 '',
 ' La Paz ',
 'La Paz',
 'La Paz and Sucre',
 'Sucre (Constitutional) and La Paz (Executive)',
 'La Paz',
 'La Paz',
 'La Paz',
 'Sucre',
 'La Paz/Sucre',
 ' La Paz ',
 'La Paz',
 'La Paz',
 'Sucre',
 '',
 '',
 'La Paz and Sucre',
 ' La Paz ',
 'La Paz (executive capital) and Sucre (constitutional capital)',
 '',
 ' La Paz ',
 'La Paz',
 ' La Paz ',
 '',
 '',
 ' La Paz ']

In [39]:
ref_aug_answers_groups[0]

["<thought></thought> The passage about Mali mentions its capital, Bamako, a few times, as well as mentions several details about Mali's locations and borders. <answer></answer> Bamako",
 '<answer></answer>\n \nFirst I need to find  where the capital is mentioned in the text. It was mentioned in the following section:\n"Mali (; ), officially the Republic of Mali, is a landlocked country in West Africa. Mali is the eighth-largest country in Africa, with an area of over . The population of Mali is million. 67% of its population was estimated to be under the age of 25 in 2017. Its capital and largest city is Bamako. \n<answer>Bamako</answer>',
 '<thought>I will look through the provided information to find the capital of Mali.</thought>\n<answer>Bamako</answer>',
 '<thought> The text mentions Mali as a country in West Africa with a rich history, including the Mali Empire, which was once the wealthiest country in Africa. It also mentions the capital of Mali, which is Bamako.</thought>\n<an